In [1]:
import numpy as _np
import spinmob as _s
import matplotlib.pyplot as _plt
import os as _os
from scipy.optimize import curve_fit

In [2]:
data_box = _s.data.load(path = "problem1data_small_test.csv")


In [3]:
def exp_decay(x, gamma, amp):
    return amp*_np.exp(-x/gamma)

def correlation(data, data_average):
    data_correlations = []
    correlation_distance = _np.array(range(len(data)-1))+1
    for distance in correlation_distance:
        correlations = data[:-distance]*data[distance::]-data_average**2
        data_correlations.append(_np.average(correlations))
    return data_correlations, correlation_distance

def fit_exp_decay(data, distacnes, p0):
    popt, cov = curve_fit(exp_decay, distacnes[:2000], data[:2000], p0=p0)
    return [popt, cov]

def plot_exp_decay_fit(correlation, distances, popt, ep_str, n_or_e):
    _plt.plot(distances, correlation, label="data")
    _plt.axhline(0, linestyle="--", color="grey", alpha=0.5)
    _plt.plot(distances[:2000], exp_decay(distances[:2000], *popt), label="fit")
    _plt.title(n_or_e+" correlation for ε="+ep_str, size=35)
    _plt.ylabel(r'$\tilde{e_n}\tilde{e}_{n+d} - \langle \tilde{e}\rangle^2$', size=35)
    _plt.xlabel("d", size=35)
    _plt.tick_params(axis="both", labelsize=30)
    _plt.legend(fontsize=30)
    _plt.show()

def choose_unc(es, es_popt, ns, ns_popt):
    es_uncs = []
    ns_uncs = []
    num_trials = len(es)
    for position, block in enumerate(block_sizes):
        blocked_es = _np.reshape(es, (int(num_trials/block), block))
        block_avg_es = _np.average(blocked_es, axis=1)
        unc = _np.std(block_avg_es)/_np.sqrt(len(block_avg_es))
        es_uncs.append(unc)

        blocked_ns = _np.reshape(ns, (int(num_trials/block), block))
        block_avg_ns = _np.average(blocked_ns, axis=1)
        unc = _np.std(block_avg_ns)/_np.sqrt(len(block_avg_ns))
        ns_uncs.append(unc)

        print("E Unc: ", es_uncs[-1], "E correlation/unc: ", _np.sqrt(exp_decay(block, *es_popt))/unc)
        print("n Unc: ", ns_uncs[-1], "n correlation/unc: ", _np.sqrt(exp_decay(block, *ns_popt))/unc)
        print(" num blocks: ", len(block_avg_es), " blocksize: ", block, "choice number", position)
        print("\r\r\r")
    which_block = int(input("Which block_size?"))
    return [block_sizes[which_block], es_uncs[which_block], ns_uncs[which_block]]


In [11]:
mus, ns, es, ws = _np.array([data for data in data_box])

In [35]:
indexes = _np.nonzero(mus==list(set(mus))[0])
ns0 = ns[indexes]
_plt.plot(range(len(ns0)), ns0)

In [32]:
indexes = _np.nonzero(mus==list(set(mus))[2])
ns0 = ns[indexes]
ns0_average = _np.average(ns0)
ns0_correlation, ns0_distances = correlation(ns0, ns0_average)
p0 = [200, 30]
ns0_popt, ns0_cov = fit_exp_decay(ns0_correlation, ns0_distances, p0)
p0 = [50, 800]
ns0_popt, ns0_cov = fit_exp_decay(ns0_correlation, ns0_distances, p0)
plot_exp_decay_fit(ns0_correlation, ns0_distances, ns0_popt, "", r"$n$")
ns0_popt

array([ 5.50276846e+08, -9.46381581e-01])

In [36]:
block_size = 50
equilibriation_time = 25
exp_decay(block_size, *ns0_popt)/ns0_average*100

-24.328573136530103

In [41]:
mus_final = []
ns_final = []
ns_unc_final = []
N=20
for mu in set(mus):
    mus_final.append(mu)
    indexes = _np.nonzero(mus==mu)
    ns_temp = ns[indexes][equilibriation_time:]
    ns_final.append(_np.average(ns_temp))
    ns_unc_final.append(_np.std(ns_temp) / _np.sqrt(len(ns_temp)/block_size))
sort_indexes = _np.argsort(mus_final)
mus_final = _np.array(mus_final)[sort_indexes]
ns_final = _np.array(ns_final)[sort_indexes]
ns_unc_final = _np.array(ns_unc_final)[sort_indexes]
print(mus_final)
print(ns_final)
print(ns_unc_final)

[0.   0.25 0.5 ]
[0.50666667 2.         4.        ]
[0.408212 0.       0.      ]


In [42]:
_plt.errorbar(mus_final, ns_final, yerr=ns_unc_final, marker=".", color="blue", zorder=1)
_plt.title(r"$n \: vs \: \mu \: at \: N=4, \: \beta=100, \epsilon=0.01$", size=18)
_plt.xlabel(r"$\beta$", size=18)
_plt.ylabel(r"$n$", size=18)
_plt.legend()

No handles with labels found to put in legend.


In [11]:
chis_unc/chis*100

array([54.96799598, 94.42488076, 37.516294  , 26.60732319, 15.79390449,
       11.56889627,  7.69512487,  6.34824015,  5.34790965,  5.2146382 ,
        4.95156347,  5.02092424,  5.01488951,  4.90754014,  4.95518506])

In [12]:
for jj in range(len(chis)):
    print(betas_final[jj], chis[jj], chis_unc[jj])

0.75 0.00412 0.002264681434551006
0.775 0.001912006321490562 0.0018054096892704218
0.8 0.00838720608052631 0.0031465688919556007
0.825 0.017815580962482924 0.004740249204437283
0.85 0.05427011183033771 0.008571369630605234
0.875 0.11166822459615378 0.012918781067399672
0.9 0.30659283188035263 0.023592701265924724
0.925 0.5169098470292232 0.03281467842691488
0.95 0.9357766803016367 0.05004449136966139
0.975 1.2070320504675711 0.06294235438385021
1.0 1.5386741561593251 0.07618842747371263
1.025 1.7865229573588368 0.08969996425442045
1.05 2.0175173894353673 0.10117626794278058
1.075 2.256677930311366 0.1107473751545197
1.1 2.4793002040418837 0.1228539133345776
